# Обработка выбросов разными способами 

In [1]:
from python_files.data import data
from python_files.matrix.mat_col_list import mat_col_list_norm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
data_ = data.copy()

##### Признак 'Угол нашивки, град', имеющий всего два уникальных значения, обрабатывать на выбросы не имеет смысла. Остальные признаки, за исключением целевого, будем обрабатывать исходя из идеи сохранения их исходного распределения.

Объявим необходимые пользовательские функции.

In [2]:
# Тип вычисления верхней и нижней границы выбросов
from enum import Enum
class OutlierBoundaryType(Enum):
    SIGMA = 1
    QUANTILE = 2
    IQR = 3

In [3]:
# Функция вычисления верхней и нижней границы выбросов
def get_outlier_boundaries(df, col, outlier_boundary_type: OutlierBoundaryType):
    if outlier_boundary_type == OutlierBoundaryType.SIGMA:
        K1 = 3
        lower_boundary = df[col].mean() - (K1 * df[col].std())
        upper_boundary = df[col].mean() + (K1 * df[col].std())

    elif outlier_boundary_type == OutlierBoundaryType.QUANTILE:
        lower_boundary = df[col].quantile(0.025)
        upper_boundary = df[col].quantile(0.975)

    elif outlier_boundary_type == OutlierBoundaryType.IQR:
        K2 = 1.5
        IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
        lower_boundary = df[col].quantile(0.25) - (K2 * IQR)
        upper_boundary = df[col].quantile(0.75) + (K2 * IQR)

    else:
        raise NameError('Unknown Outlier Boundary Type')
        
    return lower_boundary, upper_boundary    

#### 1. Удаление выбросов 

In [4]:
# Разделим исходный датасет на обучающую и тестовую выборки и будем работать с выбросами только на обучающей выборке
y = data_.pop('Соотношение матрица-наполнитель')
X = data_

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 7)

In [6]:
# соберем всю обучающую выборку в новую переменную df_train 
df_train = X_train
df_train['Соотношение матрица-наполнитель'] = y_train

##### 1.1 Удаление выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [10]:
# сначала обработаем выбросы в признаке со скошенным исходным распределением ('Поверхностная плотность, г/м2')
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением  
lower_boundary_, upper_boundary_ = get_outlier_boundaries(df_train, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)

# Флаги для удаления выбросов в признаке со скошенным исходным распределением 
outliers_ = np.where(df_train['Поверхностная плотность, г/м2'] > upper_boundary_, True, 
                     np.where(df_train['Поверхностная плотность, г/м2'] < lower_boundary_, True, False))

# Удаление выбросов на основе флага в признаке со скошенным исходным распределением
df_train_trimmed = df_train.loc[~(outliers_), ]

##### 1.2 Удаление выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [11]:
# удалим выбросы в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:

    # Вычисление по правилу "3 сигм" верхней и нижней границы 
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_trimmed, col, OutlierBoundaryType.SIGMA)
    
    # Флаги для удаления выбросов 
    outliers = np.where(df_train_trimmed[col] > upper_boundary, True, 
                        np.where(df_train_trimmed[col] < lower_boundary, True, False))
    
    # Удаление выбросов на основе флага
    df_train_trimmed_3S = df_train_trimmed.loc[~(outliers), ]

In [12]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_trimmed_3S = df_train_trimmed_3S.pop('Соотношение матрица-наполнитель')
X_train_trimmed_3S = df_train_trimmed_3S

# формируем DataFrame на основе массива
y_train_trimmed_3S = pd.DataFrame(y_train_trimmed_3S, columns=['Соотношение матрица-наполнитель'])

##### 1.3 Удаление выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [13]:
# удалим выбросы в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:

    # Вычисление по квантилям (2.5% и 97.5%) верхней и нижней границы 
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_trimmed, col, OutlierBoundaryType.QUANTILE)
    
    # Флаги для удаления выбросов 
    outliers = np.where(df_train_trimmed[col] > upper_boundary, True, 
                        np.where(df_train_trimmed[col] < lower_boundary, True, False))
    
    # Удаление выбросов на основе флага
    df_train_trimmed_QT = df_train_trimmed.loc[~(outliers), ]

In [14]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_trimmed_QT = df_train_trimmed_QT.pop('Соотношение матрица-наполнитель')
X_train_trimmed_QT = df_train_trimmed_QT

# формируем DataFrame на основе массива
y_train_trimmed_QT = pd.DataFrame(y_train_trimmed_QT, columns=['Соотношение матрица-наполнитель'])

#### 2. Замена выбросов граничными значениями

##### 2.1 Замена выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [15]:
df_train_sub = df_train.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_sub['Поверхностная плотность, г/м2'] = np.where(df_train_sub['Поверхностная плотность, г/м2'] > upper_boundary, upper_boundary, 
                np.where(df_train_sub['Поверхностная плотность, г/м2'] < lower_boundary, lower_boundary, df_train_sub['Поверхностная плотность, г/м2']))

##### 2.2 Замена выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [16]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub, col, OutlierBoundaryType.SIGMA)
    # Изменение данных
    df_train_sub[col] = np.where(df_train_sub[col] > upper_boundary, upper_boundary, 
                        np.where(df_train_sub[col] < lower_boundary, lower_boundary, df_train_sub[col]))
# cохраним результат в новую переменную
df_train_sub_bound_3S = df_train_sub

In [17]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_sub_bound_3S = df_train_sub_bound_3S.pop('Соотношение матрица-наполнитель')
X_train_sub_bound_3S = df_train_sub_bound_3S

# формируем DataFrame на основе массива
y_train_sub_bound_3S = pd.DataFrame(y_train_sub_bound_3S, columns=['Соотношение матрица-наполнитель'])

##### 2.3 Замена выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [18]:
# повторяем обработку выбросов для признака 'Поверхностная плотность, г/м2' сохранив результат в нлвую переменную 'df_train_sub_'
df_train_sub_ = df_train.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_sub_['Поверхностная плотность, г/м2'] = np.where(df_train_sub_['Поверхностная плотность, г/м2'] > upper_boundary, upper_boundary, 
            np.where(df_train_sub_['Поверхностная плотность, г/м2'] < lower_boundary, lower_boundary, df_train_sub_['Поверхностная плотность, г/м2']))

In [19]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_, col, OutlierBoundaryType.QUANTILE)
    # Изменение данных
    df_train_sub_[col] = np.where(df_train_sub_[col] > upper_boundary, upper_boundary, 
                        np.where(df_train_sub_[col] < lower_boundary, lower_boundary, df_train_sub_[col]))
# cохраним результат в новую переменную
df_train_sub_bound_QT = df_train_sub_

In [20]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_sub_bound_QT = df_train_sub_bound_QT.pop('Соотношение матрица-наполнитель')
X_train_sub_bound_QT = df_train_sub_bound_QT

# формируем DataFrame на основе массива
y_train_sub_bound_QT = pd.DataFrame(y_train_sub_bound_QT, columns=['Соотношение матрица-наполнитель'])

#### 3. Замена выбросов по медиане

##### 3.1 Замена выбросов определенных на основе межквартильного интервала в признаке со скошенным исходным распределением

In [21]:
# В признаке со скошенным исходным распределением заменим выбросы по моде
df_train_sub_med = df_train.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_med, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_sub_med['Поверхностная плотность, г/м2'] = np.where(df_train_sub_med['Поверхностная плотность, г/м2'] > upper_boundary,
                                                             df_train_sub_med['Поверхностная плотность, г/м2'].mode(), 
        np.where(df_train_sub_med['Поверхностная плотность, г/м2'] < lower_boundary, df_train_sub_med['Поверхностная плотность, г/м2'].mode(),
                    df_train_sub_med['Поверхностная плотность, г/м2']))

##### 3.2 Замена выбросов определенных по правилу "3 сигм" в признаках с исходным распределением близким к нормальному 

In [22]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_med, col, OutlierBoundaryType.SIGMA)
    # Изменение данных
    df_train_sub_med[col] = np.where(df_train_sub_med[col] > upper_boundary, df_train_sub_med[col].median(), 
                        np.where(df_train_sub_med[col] < lower_boundary, df_train_sub_med[col].median(), df_train_sub_med[col]))
# cохраним результат в новую переменную
df_train_sub_med_3S = df_train_sub_med

In [23]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_sub_med_3S = df_train_sub_med_3S.pop('Соотношение матрица-наполнитель')
X_train_sub_med_3S = df_train_sub_med_3S

# формируем DataFrame на основе массива
y_train_sub_med_3S = pd.DataFrame(y_train_sub_med_3S, columns=['Соотношение матрица-наполнитель'])

##### 3.3 Замена выбросов определенных по квантилям (2.5% и 97.5%) в признаках с исходным распределением близким к нормальному 

In [24]:
# В признаке со скошенным исходным распределением заменим выбросы по моде
df_train_sub_med_ = df_train.copy()     
# Вычисление по межквартильному интервалу верхней и нижней границы в признаке со скошенным исходным распределением
lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_med_, 'Поверхностная плотность, г/м2', OutlierBoundaryType.IQR)
# Изменение данных
df_train_sub_med_['Поверхностная плотность, г/м2'] = np.where(df_train_sub_med_['Поверхностная плотность, г/м2'] > upper_boundary,
                                                             df_train_sub_med_['Поверхностная плотность, г/м2'].mode(), 
        np.where(df_train_sub_med_['Поверхностная плотность, г/м2'] < lower_boundary, df_train_sub_med_['Поверхностная плотность, г/м2'].mode(),
                    df_train_sub_med_['Поверхностная плотность, г/м2']))

In [25]:
# Замена выбросов граничными значениями в признаках с исходным распределением близким к нормальному  
for col in mat_col_list_norm:    
    # Вычисление верхней и нижней границы
    lower_boundary, upper_boundary = get_outlier_boundaries(df_train_sub_med_, col, OutlierBoundaryType.QUANTILE)
    # Изменение данных
    df_train_sub_med_[col] = np.where(df_train_sub_med_[col] > upper_boundary, df_train_sub_med_[col].median(), 
                        np.where(df_train_sub_med_[col] < lower_boundary, df_train_sub_med_[col].median(), df_train_sub_med_[col]))
# cохраним результат в новую переменную
df_train_sub_med_QT = df_train_sub_med_

In [26]:
# выделим целевой признак в отдельную переменную после обработки выбросов
y_train_sub_med_QT = df_train_sub_med_QT.pop('Соотношение матрица-наполнитель')
X_train_sub_med_QT = df_train_sub_med_QT

# формируем DataFrame на основе массива
y_train_sub_med_QT = pd.DataFrame(y_train_sub_med_QT, columns=['Соотношение матрица-наполнитель'])